In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

# Basics of the Wireless Communication
In order to explain the basics of the wireless communication we will try to build a mock-up mobile communication system from scratch. The simple system should transmit some data from a transmiting station to a handheld device. Some basic constraints we set will be:

- the power that we use at the transmitting device is $P_{tx}=1W$
- the receiving device is at the distance of $d=1000m$


In [ ]:
d=1000  # m - the distance from the base station to the receiving device
P_tx=1  # W - the power used in the transmitting device

## History
The phenomenon that electromagnetic waves propagate through free space was already observed in 18th century and mathematically descibed by James Maxwell in 1864. This phenomenon was further explored by many scientists trying to use it for power transmission and communication. In 1894 the British physicist Oliver Lodge showed he can transmit and receive the electromagnetic waves at distances up to 50 meters. The inventor Guglielmo Marconi worked on the the idea of building a wireless telegraphy system, gained a patent on the system in 1896 and developed it into a commercial communication system over the next few years.

## Antenna

It is rather simple to make a device that will transmit the electromagnetic waves into the air. An AC voltage source connected to a pair of wires known and the "dipole antenna" will do the trick. The same dipole antenna can be used also to recieve the electromagnetic waves at some distance and convert them back into the electrical signals.

![alt text](images/Dipole_antenna.gif)


In practice only a part of the used electrical power will be converted into the electromagnetic waves. The actual power we get into the air can be denoted by $e_{antenna}P_{tx}$ where $e_{antenna}$ is a constant between 0 and 1 that describes the efficiency of the antenna to convert the electrical signal into the electromagmetic waves.

An efficient dipole anteanna with $e_{antenna}$ close to $1$ can be realized if the length of the dipole is $\lambda/2$ where $\lambda$ is the wavelength of the electromagnetic wave. For typical sinusidual wave of frequency $f_0$ the wavelength is calculated as $\lambda=c/f_0$.

In [ ]:
c   = 299792458 # m/s - speed of light (and the electromagnetic wave propagation) 
f_0 = 1e9       # Hz - operating frequeincy or the carrier frequency
Lambda = c/f_0  # m - wavelenght
print("Signal half wavelenght = {0:0.2f} cm".format(100*Lambda/2))

Besides the dipole there are many ways for designing antennas but a rule of thumb is that the antenna physiscal size should be close to $\lambda/2$. As one of the requirements is to have a handheld device this will put some constraints on the wave frequencies we can use. For example for $f_0=1GHz$ we get $\lambda/2\approx 15cm$ which is approximatelly the size of a normal handheld device. Having lower frequencies would probably result in inefficient antennas. The efficiency 

Having efficient antennas integrated into the mobile phones is actually one of the toughest design challenges of the phone manufactureres. Here are some examples of vary clever designs where anteannas are integrated in the phone rim and the back cover.

![alt text](images/Phone_antenna.jpg)


## Power estimation

The total power spent in the transmiter $P_{tx}$ is also influenced by the electronic. Additional efficiency factor $e_{electronics}$ is added as a constant between $0$ and $1$ that describes the efficiency of the electronics used to generate the signals.

If the antenna is transmitting uniformly in all directions the radiated power will be distributed uniformly over a sphere and we can calculate the power density at distance $d$ as:

$$\frac{e_{electronics}e_{antenna}P_{tx}}{4\pi d^2} [W/m^2]$$

This is bad as the energy is getting spread over a large area. But it gets even worse as a small mobile device antenna will gather the power only form a small part of the space. 

One way to describe the receiver antenna efficiency is by sa called antenna effective area denoted as $A_{rx}$ which is typically smaller than the physical size. Using this generalization a simple formula to calculate the received power is: 

$$ P_{rx}= A_{rx} \frac{e_{electronics}e_{antenna}P_{tx}}{4\pi d^2}$$

We can put some realistic numbers to get an estimate of the transmitted power for our case. We set $e_{tx}=e_{electronics}e_{antenna}=0.5*0.5$ and $A_{rx}=0.1m*0.1m = 0.01m^2$.

In [ ]:
e_tx=0.5*0.5 # efficiency of the transmitter
A_rx=0.1*0.1 # m2 -receiver antenna effective area

P_rx=A_rx*e_tx*P_tx/(4*np.pi*d**2)

print("Received signal power = {} W".format(P_rx))

This is very small power and would be useless as a power source. However, if we somehow embbed some information in the signal than the modern electronics would be able to detect the signal and extract the information easily as we will show next.

The simplified calculation did not take into account directiveness of the antennas which can be described as an additional antenna gain factor. Also we did not take the attenuation of the signals in the air:   

![alt text](images/Atmospheric_electromagnetic_opacity.svg)


## Milimeter Waves

For very high frequencies the wavelength becomes short, in the range of milimeters, and the antennas can be small. For example for 60 GHz frequency the wavelength becomes 5mm. Here is an image of an antenna array inteneded for 60GHz operation with 12 antennas:

![alt text](images/mmWave_antenna.jpg)

Having small antennas is very usefull and enables various multientenna processing solutions. However, working with the high frequency signals becomes challenging for designing electronics. In addition, the attenuation in the air becomes worse at the higher frequencies. Finally, the milimeter waves will not pass through most of the materials, e.g. building walls and also get reflected easily making design of a communication system quite different.


## Signal modulation 

Having covered the basics of electomagnetic waves we also calcualted that the power level at the receiver will be extremely low. Next we will try to use this low power signal for transfering some information. We will start immediately with digital information in form of bits which we want to somehow embbed into the signal. 

In [ ]:
# generate some random data
bits = np.random.binomial(n=1, p=0.5, size=512)
print('Number of bits : {} , the first 4 bits: {}'.format(len(bits),bits[0:4]))

Embedding the information is done by changing or modulating the signal usualy called signal modulation. In our case we will try the simplest form of modulation where we just multiply the signal by the bits. This is also known as "on-off keying". Let the duration of each bit will deterine the speed of communication.

In [ ]:
T=10*1e-9     # s - duration of one bit during transmission
print('Communcation speed: {} Mbits/s'.format(1e-6/T))

Ts=1/(4*f_0)  # s - sampling period in seconds - high sampling rate for visualization purposes

# generate the modulated signal
s_tx = lambda t: bits[np.floor_divide(t,T).astype('int')]*np.cos(2*np.pi*f_0*t)

# show the first 4 bits
t=np.arange(0,4*T,Ts)
plt.plot(t*1e9,s_tx(t))
plt.xlabel('Time[ns]')
plt.title('Modulated signal - the first 4 bits: {}'.format(bits[0:4]))
plt.show()

## Modelling the Recieved Signal

The transmitted signal will be arrive atennuated at the receiver. It will also be corrupted by various noise and interference sources. In ideal case we will have only the noise of the electronics of the receiver. A clever design of the receiver will limit the noise only to the thermal noise. Assuming that we properly filter the signal and sample with time interval $T_s$ we can approximate the thermal noise and the received signal as:

In [ ]:
kB = 1.3806488e-23  #J/K - Boltzmann constant - particle energy with temperature
Ta = 273+50         # K  - temperature of the electronics

P_thermal_noise=4*kB*Ta*1/Ts

SNR=P_rx/P_thermal_noise

s_rx=lambda t: s_tx(t)+np.sqrt(0.5/SNR)*np.random.randn(len(t))

# show the first 4 bits
t=np.arange(0,4*T,Ts)
plt.plot(t*1e9,s_rx(t))
plt.xlabel('Time[ns]')
plt.title('Modulated signal - the first 4 bits: {}'.format(bits[0:4]))
plt.show()

## Detection

Next step is to try to extract the embedded information from the noisy signal. Optimal detection for our signal will be achieved if we can generate a sinusoidal signal in the receiver which is perfectly alligned with the signal used in the transmitter.

The received signal is typically first multiplied by the locally generated sinusodal signal using analog ciscuits. We can model this as below:

In [ ]:
# assumptions on the receiver generated sinusoidal signal
f_rx=f_0
phase_rx=0

s_rx_dem=lambda t: s_tx(t)*np.cos(2*np.pi*f_rx*t+phase_rx)+np.sqrt(0.5/SNR)*np.random.randn(len(t))

# show the first 4 bits
t=np.arange(0,4*T,Ts)
plt.plot(t*1e9,s_rx_dem(t))
plt.xlabel('Time[ns]')
plt.title('Demodulated signal - the first 4 bits: {}'.format(bits[0:4]))
plt.show()

As we decided to to oversample the signal we can then also average the samples that should belong to the same bit. Averaging is a form of a digital low pass flter. Note that we now also need to have the bit timing also alligned with the transmitter.

In [ ]:
N_oversampling = np.round(T/Ts).astype('int')
bitsE=lambda t: np.mean(s_rx_dem(t).reshape(-1,N_oversampling),axis=1)

# show the first 4 bits
t=np.arange(0,4*T,Ts)
plt.plot(t*1e9,s_rx_dem(t))
plt.plot(t[N_oversampling//2::N_oversampling]*1e9,bitsE(t),'o')
plt.xlabel('Time[ns]')
plt.title('Demodulated signal - the first 4 bits: {} and values {}'.format(bits[0:4],bitsE(t)))
plt.show()

Now lets simulate this for many bits and see how the values for the bits are distributed.

In [ ]:
res=bitsE(np.arange(0,len(bits)*T,Ts))

plt.hist(res[bits==1],density=True)
plt.hist(res[bits==0],density=True)
plt.show()

print("sigma = {}, sigma estimated = {}".format(np.sqrt(0.5/(N_oversampling*SNR)),np.std(res[bits==1])))

## Bit Error Rate

As seen above, depending on the noise, some bits might be misinterpreted and flipped at the receiver. The probablioty of the error, also known as BER (Bit Error Rate) is calculated as:

$$p_{error}=p(0|1)p_1+p(1|0)p_0$$

In typical random data the probability of a bit being a zero or one is equal $p_1=p_0=0.5$.

In out case the result can be approximated by a Gaussian function and the BER calculated as:

$$p_{error}=0.5 erfc(\sqrt{SNR})$$

In [ ]:
import scipy.special as sc

p_error=0.5*sc.erfc(np.sqrt(N_oversampling*SNR)/4)

print("Bit error rate = {}".format(p_error))

As we saw above the singal to noise ratio will be inversly proportional to the communication speed. Longer bits and slower communication will lead to better signal to noise ratios.

In [ ]:
B = np.linspace(5*10**7,10**8,100)
p_error=0.5*sc.erfc(np.sqrt(N_oversampling*SNR/(T*B))/4)

fig, ax = plt.subplots()
plt.plot( B/10**6, p_error )
ax.set_yscale('log')
plt.xlabel('Mbits/s')
plt.ylabel('bit error rate')
plt.show()